In [ ]:
%pylab inline
import torch
import sys, os
import pystk
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)

In [ ]:
class Rollout:
    def __init__(self, screen_width, screen_height, hd=True, track='lighthouse', render=True):
        # Init supertuxkart
        if not render:
            config = pystk.GraphicsConfig.none()
        elif hd:
            config = pystk.GraphicsConfig.hd()
        else:
            config = pystk.GraphicsConfig.ld()
        config.screen_width = screen_width
        config.screen_height = screen_height
        pystk.init(config)
        
        self.render = render
        race_config = pystk.RaceConfig(track=track)
        self.race = pystk.Race(race_config)
        self.race.start()
    
    def __call__(self, agent, n_steps=200):
        self.race.restart()
        self.race.step()
        data = []
        track_info = pystk.Track()
        track_info.update()

        for i in range(n_steps):
            world_info = pystk.WorldState()
            world_info.update()

            # Gather world information
            kart_info = world_info.players[0].kart

            agent_data = {'track_info': track_info, 'kart_info': kart_info}
            if self.render:
                agent_data['image'] = np.array(self.race.render_data[0].image)

            # Act
            action = agent(**agent_data)
            agent_data['action'] = action

            # Take a step in the simulation
            self.race.step(action)

            # Save all the relevant data
            data.append(agent_data)
        return data

In [ ]:
def dummy_agent(**kwargs):
    action = pystk.Action()
    action.acceleration = 1
    return action

In [ ]:
rollout = Rollout(800, 600)

def rollout_many(many_agents, **kwargs):
    for agent in many_agents:
        yield rollout(agent, **kwargs)

data = rollout(dummy_agent)

In [ ]:
def show_video(frames, fps=30):
    import imageio
    from IPython.display import Video, display 
    
    imageio.mimwrite('/tmp/test.mp4', frames, fps=fps, bitrate=10000000)
    display(Video('/tmp/test.mp4', width=800, height=600, embed=True))

show_video([d['image'] for d in data])

In [ ]:
def show_agent(agent, **kwargs):
    data = rollout(agent, **kwargs)
    show_video([d['image'] for d in data])

In [ ]:
def less_dummy_agent(**kwargs):
    action = pystk.Action()
    action.acceleration = 1
    action.steer = -0.6
    return action
show_agent(less_dummy_agent)

In [ ]:
# Let's load a fancy auto-pilot. You'll write one yourself in your homework.
from _auto_pilot import auto_pilot

show_agent(auto_pilot, n_steps=600)